In [ ]:
import re
import random
import requests
import gradio as gr

class ChatbotCuaca:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = "http://api.weatherapi.com/v1"
        self.url_current = "{base}/current.json?key={api_key}&q={city}&lang=id"
        self.url_forecast = "{base}/forecast.json?key={api_key}&q={city}&days={days}&lang=id"

        # Pola pesan yang dikenali bot
        self.pola_respons = {
            r'(besok|esok)( (cuaca|prakiraan|ramalan|forecast))?( di (?P<kota>.+))?( gimana| bagaimana)?': '__CUACA_BESOK__',
            r'(cuaca|prakiraan|ramalan|forecast)( besok| esok)( di (?P<kota>.+))?( gimana| bagaimana)?': '__CUACA_BESOK__',
            r'(rekomendasi|tempat|ide|info) (wisata|jalan[- ]jalan)( di (?P<kota>.+))?': '__REKOMENDASI_WISATA__',
            r'(cuaca|prakiraan|ramalan|suhu|forecast)( hari ini)?( di (?P<kota>.+))?': '__CUACA_HARI_INI__',
            r'(rekomendasi|ide|enaknya ngapain) kegiatan( di (?P<kota>.+))?': '__REKOMENDASI_KEGIATAN__',
            r'^(halo|hai|hello|hi)$': 'Halo! Saya Chatbot Wisata & Cuaca ☀️ Siap bantu kamu jalan-jalan seru! 😎',
            r'(terima kasih|thanks|makasih)': 'Sama-sama! Jangan lupa bawa payung kalau mendung ya ☁️',
            r'(bye|selamat tinggal|dadah)': 'Sampai jumpa lagi! Semoga liburanmu menyenangkan ✨',
        }

    def get_emoji(self, kondisi):
        kondisi = kondisi.lower()
        if "rain" in kondisi or "hujan" in kondisi:
            return "🌧️"
        elif "sunny" in kondisi or "cerah" in kondisi:
            return "☀️"
        elif "cloud" in kondisi or "berawan" in kondisi:
            return "⛅"
        elif "fog" in kondisi or "kabut" in kondisi:
            return "🌫️"
        else:
            return "🌤️"

    def ambil_cuaca_hari_ini(self, kota):
        try:
            url = self.url_current.format(base=self.base_url, api_key=self.api_key, city=kota)
            response = requests.get(url, timeout=5)
            data = response.json()
            if response.status_code == 200 and 'current' in data:
                kondisi = data['current']['condition']['text']
                suhu = data['current']['temp_c']
                feels_like = data['current']['feelslike_c']
                kelembapan = data['current']['humidity']
                emoji = self.get_emoji(kondisi)
                tambahan = ""
                if "hujan" in kondisi.lower() or "rain" in kondisi.lower():
                    tambahan = "Jangan lupa bawa payung ya ☂️"
                elif "cerah" in kondisi.lower() or "sunny" in kondisi.lower():
                    tambahan = "Jangan lupa minum air putih yang cukup supaya tidak dehidrasi 💧"
                return (
                    f"{emoji} Cuaca hari ini di {kota.title()}:\n"
                    f"- Kondisi: {kondisi}\n"
                    f"- Suhu: {suhu}°C (terasa seperti {feels_like}°C)\n"
                    f"- Kelembapan: {kelembapan}%\n"
                    f"{tambahan}"
                )
            else:
                return f"Maaf, data cuaca untuk kota '{kota.title()}' tidak tersedia. 😥"
        except Exception as e:
            return f"❌ Terjadi kesalahan saat mengambil data cuaca: {e}"

    def ambil_cuaca_besok(self, kota):
        try:
            url = self.url_forecast.format(base=self.base_url, api_key=self.api_key, city=kota, days=2)
            response = requests.get(url, timeout=5)
            data = response.json()
            if response.status_code == 200 and 'forecast' in data:
                besok = data['forecast']['forecastday'][1]['day']
                kondisi = besok.get('condition', {}).get('text', 'Tidak diketahui')
                suhu_min = besok.get('mintemp_c', 'N/A')
                suhu_max = besok.get('maxtemp_c', 'N/A')
                hujan = besok.get('daily_chance_of_rain', 'N/A')
                emoji = self.get_emoji(kondisi)
                if "hujan" in kondisi.lower():
                    tambahan = "Besok ada kemungkinan hujan, pakai jaket biar nggak kedinginan 🧥"
                elif "cerah" in kondisi.lower():
                    tambahan = "Jangan lupa pakai sunscreen & topi biar nggak kepanasan 🧢☀️"
                else:
                    tambahan = "Cek agenda besok dan sesuaikan dengan kondisi cuaca 🗓️"
                return (
                    f"{emoji} Prakiraan cuaca besok di {kota.title()}:\n"
                    f"- Kondisi: {kondisi}\n"
                    f"- Suhu: {suhu_min}°C – {suhu_max}°C\n"
                    f"- Peluang hujan: {hujan}%\n"
                    f"{tambahan}"
                )
            else:
                return f"Maaf, tidak bisa mengambil data cuaca besok untuk kota '{kota.title()}'. 😥"
        except Exception as e:
            return f"❌ Terjadi kesalahan saat mengambil data cuaca besok: {e}"

    def rekomendasi_wisata(self, kota):
        try:
            url = self.url_current.format(base=self.base_url, api_key=self.api_key, city=kota)
            response = requests.get(url, timeout=5)
            data = response.json()
            if response.status_code == 200 and 'current' in data:
                kondisi = data['current']['condition']['text']
                emoji = self.get_emoji(kondisi)
                if "hujan" in kondisi.lower():
                    return (
                        f"{emoji} Karena cuaca saat ini di {kota.title()} sedang {kondisi.lower()}, "
                        "lebih cocok wisata indoor seperti:\n"
                        "🏛️ Museum atau galeri seni\n"
                        "☕ Nongkrong santai di kafe\n"
                        "🛍️ Jalan-jalan di mall"
                    )
                else:
                    return (
                        f"{emoji} Cuaca saat ini di {kota.title()} sedang {kondisi.lower()}, "
                        "pas banget untuk wisata outdoor seperti:\n"
                        "🌳 Taman kota atau kebun raya\n"
                        "🏖️ Pantai atau danau\n"
                        "📸 Hunting foto di spot instagramable"
                    )
            else:
                return f"Maaf, tidak bisa mengambil data cuaca untuk kota '{kota.title()}'. 😥"
        except Exception as e:
            return f"❌ Terjadi kesalahan saat mengambil rekomendasi wisata: {e}"

    def rekomendasi_kegiatan(self, kota):
        try:
            url = self.url_current.format(base=self.base_url, api_key=self.api_key, city=kota)
            data = requests.get(url, timeout=5).json()
            kondisi = data['current']['condition']['text']
            emoji = self.get_emoji(kondisi)
            if "hujan" in kondisi.lower():
                return (
                    f"{emoji} Cuaca di {kota.title()} saat ini sedang {kondisi.lower()}.\n"
                    "Berikut beberapa rekomendasi kegiatan seru saat hujan:\n"
                    "☕ Menikmati kopi atau teh hangat di kafe\n"
                    "🎬 Menonton film atau series favorit di rumah\n"
                    "📚 Membaca buku atau komik\n"
                    "🍰 Membuat camilan atau dessert sederhana"
                )
            else:
                return (
                    f"{emoji} Cuaca di {kota.title()} sedang {kondisi.lower()}.\n"
                    "Berikut beberapa rekomendasi kegiatan outdoor yang seru:\n"
                    "🚶‍♂️ Jalan-jalan santai di taman\n"
                    "🍧 Kulineran atau mencoba street food lokal\n"
                    "🏞️ Mengunjungi objek wisata alam\n"
                    "🚴‍♀️ Bersepeda santai keliling kota"
                )
        except Exception as e:
            return f"❌ Terjadi kesalahan saat mengambil rekomendasi kegiatan: {e}"


    def respons_default(self):
        return random.choice([
            "Coba tanya: 'cuaca di Bandung', atau 'rekomendasi wisata di Yogyakarta' 😊",
            "Saya bisa bantu info cuaca, sunrise, sunset, dan ide jalan-jalan. Coba aja ketik: 'rekomendasi kegiatan di Jakarta' ☔"
        ])

# Buat bot
API_KEY = 'aaf0a88742ff46cea0f215436250507'
bot = ChatbotCuaca(API_KEY)

# Fungsi handler untuk Gradio
def chatbot_response(pesan):
    pesan = pesan.strip().lower()
    if not pesan:
        return "🤖 Bot: Ketik pertanyaan dulu ya 😊"
    for pola, respons in bot.pola_respons.items():
        cocok = re.search(pola, pesan)
        if cocok:
            kota = cocok.group('kota') or 'Jakarta'
            kota = kota.strip()
            if respons == '__CUACA_HARI_INI__':
                return bot.ambil_cuaca_hari_ini(kota)
            elif respons == '__CUACA_BESOK__':
                return bot.ambil_cuaca_besok(kota)
            elif respons == '__REKOMENDASI_WISATA__':
                return bot.rekomendasi_wisata(kota)
            elif respons == '__REKOMENDASI_KEGIATAN__':
                return bot.rekomendasi_kegiatan(kota)
            else:
                return respons
    return bot.respons_default()

with gr.Blocks(css="""
body {
    background: url('https://images.unsplash.com/photo-1507525428034-b723cf961d3e?fit=crop&w=1350&q=80') no-repeat center center fixed;
    background-size: cover;
    font-family: 'Segoe UI', sans-serif;
    color: #fff;
    margin: 0;
    padding: 0;
}

.gradio-container {
    background-color: rgba(0, 0, 0, 0.5);
    border-radius: 16px;
    padding: 30px;
    max-width: 800px;
    margin: auto;
    box-shadow: 0 8px 20px rgba(0,0,0,0.4);
}

h1 {
    text-align: center;
    font-size: 28px;
    margin: 10px 0;
}

p {
    color: #e0f7fa;
    text-align: center;
    font-size: 16px;
}

button {
    background-color: rgba(255,255,255,0.2);
    color: #fff;
    border: 1px solid #fff;
    border-radius: 8px;
    padding: 8px 12px;
    font-size: 14px;
    cursor: pointer;
    transition: all 0.3s ease;
}

button:hover {
    background-color: rgba(255,255,255,0.4);
    color: #000;
}

textarea {
    border-radius: 8px !important;
    border: none !important;
}

/* Footer style */
.footer {
    text-align: center;
    margin-top: 15px;
    font-size: 13px;
    color: #d0e8ea;
}
""") as demo:

    gr.Image(
        value="https://cdn-icons-png.flaticon.com/512/1163/1163661.png",
        elem_id="logo",
        show_label=False,
        height=80
    )

    gr.Markdown(
        """
        # 🤖 **Chatbot Cuaca & Wisata Indonesia**
        Selamat datang! 🌤️
        Saya siap membantu memberikan info cuaca real-time, prakiraan besok, dan rekomendasi wisata atau kegiatan sesuai kondisi cuaca.

        **Contoh pertanyaan yang bisa kamu coba:**
        - `Cuaca hari ini di Bandung?`
        - `Besok cuaca di Jakarta gimana?`
        - `Rekomendasi wisata di Yogyakarta`
        - `Rekomendasi kegiatan di Bali`
        - `Halo`

        _Tulis pertanyaanmu di bawah lalu klik tombol Kirim!_ 😊☀️🌧️
        """
    )

    input_box = gr.Textbox(
        lines=2,
        placeholder="Tulis pertanyaanmu di sini...",
        label="💬 Masukkan Pertanyaan"
    )

    submit_btn = gr.Button("🚀 Kirim Pertanyaan")

    output_box = gr.Textbox(
        label="🤖 Jawaban Chatbot"
    )

    # Event handler untuk submit manual
    input_box.submit(fn=chatbot_response, inputs=input_box, outputs=output_box)

    # Event handler untuk tombol submit
    submit_btn.click(fn=chatbot_response, inputs=input_box, outputs=output_box)

    # Footer
    gr.Markdown(
        "<div class='footer'>© 2025 Chatbot Cuaca & Wisata • Dibuat oleh Agil Deriansyah Hasan</div>"
    )

demo.launch()



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://54a850aa44475d44fa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


AttributeError: module 'gradio' has no attribute 'blocks'